In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15641941
paper_name = 'outten_culotta_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [20]:
original_data = pd.read_excel('raw_data/outten_culotta_2005.xlsx', sheet_name='Sheet1', header=None)

In [21]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 90 x 2


In [22]:
original_data.head()

,0,1
0,YBR112C,1.0
1,YDL005C,1.0
2,YGL025C,1.0
3,YHR008C,1.0
4,YIL128W,1.0


In [23]:
original_data['orf'] = original_data[0].astype(str)

In [24]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [25]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [26]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, orf]
Index: []


In [27]:
# Adjust scores such that -1 = weakest phenotype, -4 = strongest phenotype.
original_data['data'] = pd.to_numeric(original_data[1], errors='coerce') - 5

In [28]:
original_data.loc[original_data['data'].isnull(),'data'] = 0

In [29]:
original_data.set_index('orf', inplace=True)

In [30]:
original_data = original_data[['data']].copy()

In [31]:
original_data = original_data.groupby(original_data.index).mean()

In [32]:
original_data.shape

(90, 1)

# Load & process tested strains

In [33]:
tested = pd.read_excel('raw_data/Yeast Knockout -BY4741.xlsx', sheet_name='mat_a_060701.txt')

In [34]:
tested.head()

,ID,ORF,Gene,Mating type,Unnamed: 4,Plate,Row,Column,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,338.0,YAL068C,NaN,BY4741,01_1,1.0,A,2.0,NaN,NaN,NaN
1,339.0,YAL067C,NaN,BY4741,01_1,1.0,A,3.0,NaN,NaN,NaN
2,340.0,YAL066W,NaN,BY4741,01_1,1.0,A,4.0,NaN,NaN,NaN
3,341.0,YAL065C,NaN,BY4741,01_1,1.0,A,5.0,NaN,NaN,NaN
4,345.0,YAL062W,NaN,BY4741,01_1,1.0,A,6.0,NaN,NaN,NaN


In [35]:
tested['orf'] = tested['ORF'].astype(str)

In [36]:
tested['orf'] = clean_orf(tested['orf'])

In [39]:
tested.loc[tested['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [40]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [41]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             ID  ORF Gene Mating type Unnamed: 4  Plate  Row  Column  \
index_input                                                            
4710        NaN  NaN  NaN         NaN        NaN    NaN  NaN     NaN   

            Unnamed: 8 Unnamed: 9 Unnamed: 10  orf  
index_input                                         
4710               NaN        NaN         NaN  NAN  


In [42]:
tested = tested.loc[t,:]

In [43]:
tested_orfs = tested['orf'].unique()

In [44]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YDL005C']

In [45]:
tested_orfs = list(tested_orfs) + missing

In [46]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [47]:
data = original_data.copy()

In [48]:
dataset_ids = [110]
datasets = datasets.reindex(index=dataset_ids)

In [49]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [50]:
data.head()

dataset_id,110
data_type,value
orf,
YAL068C,0.0
YAL067C,0.0
YAL066W,0.0
YAL065C,0.0
YAL062W,0.0


## Subset to the genes currently in SGD

In [51]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [52]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,110
,data_type,value
gene_id,orf,
1869,YAL068C,0.0
61,YAL067C,0.0
60,YAL066W,0.0
1727,YAL065C,0.0
57,YAL062W,0.0


# Normalize

In [53]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [54]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [55]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        110       
data_type       value valuez
gene_id orf                 
1869    YAL068C   0.0    0.0
61      YAL067C   0.0    0.0
60      YAL066W   0.0    0.0
1727    YAL065C   0.0    0.0
57      YAL062W   0.0    0.0

# Print out

In [56]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [57]:
from IO.save_data_to_db3 import *

In [58]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 15641941...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.33s/it]

Updating the data_modified_on field...
